In [72]:
library(readxl)
library(tidyverse)
library(nnet)
library(lmtest)

In [73]:
# Read the Excel file
data <- read_excel("./data/exercice1.xlsx")
# Display the first few rows of the data
head(data)
str(data)

ID,sexe,Region,Age,etat_civil,N_educatif,Membre_famille,Revenu_famille,Revenu_Mensuelle,Redoublant,⋯,Niveau_educ_pere,Niveau_educ_mere,Sign_ecole,Endroit_freq,Ph_famille,Ricule_Prof,Ricule_collegue,Sensation_apres_ecole,Apres_repture_ecole,Violence
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,2,1,3,5,0,0,1,⋯,2,2,1,1,1,0,1,1,1,1
2,0,1,4,2,3,5,0,0,1,⋯,2,1,1,2,2,1,1,1,1,1
3,0,1,4,2,3,11,0,100,1,⋯,2,1,2,1,3,0,0,1,2,0
5,0,1,3,2,3,4,0,0,1,⋯,1,1,1,1,2,1,1,3,3,1
8,0,1,1,1,3,5,1,0,1,⋯,3,3,3,4,4,0,0,3,2,1
9,0,1,4,3,3,4,1,100,0,⋯,2,3,4,1,4,1,1,1,2,0


tibble [91 × 24] (S3: tbl_df/tbl/data.frame)
 $ ID                   : num [1:91] 1 2 3 5 8 9 10 11 12 15 ...
 $ sexe                 : num [1:91] 0 0 0 0 0 0 0 0 0 0 ...
 $ Region               : num [1:91] 1 1 1 1 1 1 1 1 1 1 ...
 $ Age                  : num [1:91] 2 4 4 3 1 4 4 4 4 4 ...
 $ etat_civil           : num [1:91] 1 2 2 2 1 3 3 2 2 2 ...
 $ N_educatif           : num [1:91] 3 3 3 3 3 3 3 3 3 3 ...
 $ Membre_famille       : num [1:91] 5 5 11 4 5 4 7 8 9 10 ...
 $ Revenu_famille       : num [1:91] 0 0 0 0 1 1 0 0 0 0 ...
 $ Revenu_Mensuelle     : num [1:91] 0 0 100 0 0 100 100 0 0 0 ...
 $ Redoublant           : num [1:91] 1 1 1 1 1 0 1 1 1 1 ...
 $ Prob_Enseignat       : num [1:91] 0 1 0 1 0 0 0 1 0 0 ...
 $ Aide_ecole           : num [1:91] 1 4 4 4 1 4 5 1 5 2 ...
 $ Rq_Comprtement       : num [1:91] 1 4 1 2 1 1 1 1 1 1 ...
 $ Relat_Parent         : num [1:91] 1 1 1 1 1 1 1 1 3 1 ...
 $ Niveau_educ_pere     : num [1:91] 2 2 2 1 3 2 2 1 2 2 ...
 $ Niveau_educ_mere     : nu

In [74]:
# Convert categorical variables to factors
data <- data %>%
  mutate(
    sexe = as.factor(sexe),
    Region = as.factor(Region),
    etat_civil = as.factor(etat_civil),
    Redoublant = as.factor(Redoublant),
    Prob_Enseignat = as.factor(Prob_Enseignat),
    Aide_ecole = as.factor(Aide_ecole),
    Rq_Comprtement = as.factor(Rq_Comprtement),
    Relat_Parent = as.factor(Relat_Parent),
    Sign_ecole = as.factor(Sign_ecole),
    Endroit_freq = as.factor(Endroit_freq),
    Ph_famille = as.factor(Ph_famille),
    Ricule_Prof = as.factor(Ricule_Prof),
    Ricule_collegue = as.factor(Ricule_collegue),
    Sensation_apres_ecole = as.factor(Sensation_apres_ecole),
    Apres_repture_ecole = as.factor(Apres_repture_ecole),
    Violence = as.factor(Violence)
  )
str(data)

tibble [91 × 24] (S3: tbl_df/tbl/data.frame)
 $ ID                   : num [1:91] 1 2 3 5 8 9 10 11 12 15 ...
 $ sexe                 : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...
 $ Region               : Factor w/ 2 levels "1","2": 1 1 1 1 1 1 1 1 1 1 ...
 $ Age                  : num [1:91] 2 4 4 3 1 4 4 4 4 4 ...
 $ etat_civil           : Factor w/ 4 levels "1","2","3","4": 1 2 2 2 1 3 3 2 2 2 ...
 $ N_educatif           : num [1:91] 3 3 3 3 3 3 3 3 3 3 ...
 $ Membre_famille       : num [1:91] 5 5 11 4 5 4 7 8 9 10 ...
 $ Revenu_famille       : num [1:91] 0 0 0 0 1 1 0 0 0 0 ...
 $ Revenu_Mensuelle     : num [1:91] 0 0 100 0 0 100 100 0 0 0 ...
 $ Redoublant           : Factor w/ 2 levels "0","1": 2 2 2 2 2 1 2 2 2 2 ...
 $ Prob_Enseignat       : Factor w/ 2 levels "0","1": 1 2 1 2 1 1 1 2 1 1 ...
 $ Aide_ecole           : Factor w/ 5 levels "1","2","3","4",..: 1 4 4 4 1 4 5 1 5 2 ...
 $ Rq_Comprtement       : Factor w/ 4 levels "1","2","3","4": 1 4 1 2 1 1 1 1 1 1 ...
 $ R

In [75]:
model1 <- multinom(Apres_repture_ecole ~ sexe + Age + Region + etat_civil
                   + N_educatif + Membre_famille + Revenu_famille
                   + Revenu_Mensuelle + Redoublant + Prob_Enseignat
                   + Aide_ecole + Rq_Comprtement + Relat_Parent
                   + Niveau_educ_pere + Niveau_educ_mere + Sign_ecole
                   + Endroit_freq + Ph_famille + Ricule_Prof
                   + Ricule_collegue, data = data)
summary(model1)

# weights:  152 (111 variable)
initial  value 124.766493 
iter  10 value 67.828110
iter  20 value 19.010555
iter  30 value 1.516952
iter  40 value 1.386463
iter  50 value 1.386295
final  value 1.386294 
converged


Warning message in sqrt(diag(vc)):
"NaNs produced"


Call:
multinom(formula = Apres_repture_ecole ~ sexe + Age + Region + 
    etat_civil + N_educatif + Membre_famille + Revenu_famille + 
    Revenu_Mensuelle + Redoublant + Prob_Enseignat + Aide_ecole + 
    Rq_Comprtement + Relat_Parent + Niveau_educ_pere + Niveau_educ_mere + 
    Sign_ecole + Endroit_freq + Ph_famille + Ricule_Prof + Ricule_collegue, 
    data = data)

Coefficients:
  (Intercept)    sexe1       Age    Region2 etat_civil2 etat_civil3 etat_civil4
2  -287.36754 12.04440 -25.92159 -146.68128   240.99055   217.13666   -50.47228
3   -39.35314 88.71750  31.12081   79.10214   233.52037    12.64519   167.51897
4  -125.22920 18.92821  61.97353  -33.94136    79.79206   -61.97937   -63.74664
   N_educatif Membre_famille Revenu_famille Revenu_Mensuelle Redoublant1
2   18.941663       2.568969      79.536861       0.07671753   177.09562
3   -3.239772      -6.500250      -6.532371       0.03932989   -18.93271
4 -119.612079      25.834745      35.542686       0.18250026   -79.93612
  

```markdown
Intercept (-287.37) : Valeur de la variable cible lorsque toutes les variables sont nulles (peu significative ici).

sexe1 (12.04) : Si l'individu est masculin, la variable cible augmente de 12.04.

Age (-25.92) : Chaque année supplémentaire d'âge diminue la variable cible de 25.92.

Region2 (-146.68) : L'appartenance à la région 2 diminue la variable cible de 146.68.

Etat_civil2 (240.99), Etat_civil3 (217.13), Etat_civil4 (-50.47) : L'état civil influence la variable cible, avec des effets positifs pour "Etat_civil2" et "Etat_civil3", et négatifs pour "Etat_civil4".

Redoublant1 (177.10) : Un individu ayant redoublé voit sa variable cible augmenter de 177.10.
```